---

# 빅데이터 분석을 위한 실시간 로지스틱 회귀모형에 관현 연구
## : 베이지안 접근법을 중심으로

---

## 1. 배경

###### 새로운 특성의 데이터

> - 많은 (범주형)변수
> - 유동적인 범주값, 추가적인 범주가 지속적으로 추가됨
> - 수천만건 이상의 sample size
> - 실시간에 가까운 분석 필요
> - ...

###### 새로운 특성의 데이터를 위한 분석 방법

> - 대규모의 분산 데이터 처리(Scalable Massive Distributed Data Processing)
> - **해싱을 이용한 가변수 코딩(Feature Hashing)**
> - **온라인 최적화(Stochastic Gradient Descent, Assumed Density Filtering)**
> - ...

***---***

> 우리는 데이터 폭증의 시대에 살고 있다. 웹 로그, IoT기기 로그, 게임 로그 등 단일 서비스에서 하루에 발생하는 로그의 양은 상상을 초월한다. 예를들어 facebook의 하루 로그 크기는 600TB정도로서 기존의 통계 분석 방법으로는 간단한 인사이트 조차 얻기 어려운 경우가 많다. 그리고 데이터의 규모가 커진 것 뿐만 아니라 급변하는 사용자 요구 사항과 경쟁자의 출현에 맞서 거의 실시간으로 이러한 데이터를 분석해 빠른 의사결정을 하거나 즉각적인 서비스를 제공하는 등의 대응을 해야하는 상황에 직면하고 있다. 또한 데이터의 건수가 많아지는 문제와 더불어 다수의 범주형 변수와 각 범주형 변수마다 수백 수천개의 범주가 있고 시간이 지남에 따라 예측할 수 없는 다양한 범주가 추가될 수 있는 형태의 데이터를 분석해야 하는 경우도 생겨난다.

> 예를들어 TrueSkill과 같이 다량의 플레이어 게임 메칭 데이터를 이용해 플레이어의 승률을 계산하여 최적의 게임 메칭 상태를 찾는 문제\citep{Herbrich2006}, 특정 Facebook 사용자의 timeline에 다양한 조건의 광고 중 어떤 광고를 노출 시켜야 광고 클릭 확률이 높을 것인지를 예측하는 문제\citep{He2014}, 매출의 대부분을 차지하는 고부가 가치 유저(High-Valued Player)가 게임에서 이탈할 확률을 계산하는 문제\citep{Runge2014} 등은 대규모 데이터 분석과 빠른 예측이 비즈니스의 성패를 좌우하는 경우라고 할 수 있다. 이러한 문제를 풀기 위해서는 규모 가변적이고 대규모의 분산 데이터 처리(Scalable Massive Distributed Data Processing)를 위한 하드웨어와 소프트웨어, 그리고 이에 맞는 분석 방법론이 필요하고 본 논문에서는 분석 방법론에 대해 고찰해 볼 것이다.
 
> 앞서의 상황과 같이 유동적 다변량의 다샘플 데이터를 실시간으로 분석하는 것은 기존 배치방식의 통계 분석으로는 효과적이지 않다. 가령 데이터가 스트리밍으로 유입되고 이를 분석 데이터에 추가하고 모수 값을 계산해야하는 상황에서 매번 전체 데이터에 대해 모형 적합을 수행해서는 실시간에 가까운 예측 결과를 내놓기 어렵다. 또한 새로운 변수 혹은 범주가 데이터 세트에 추가될 때마다 모수 벡터를 다시 구성하고 모형 적합을 다시 수행하는 것은 좋은 접근법이라 할 수 없다. 

> 본 논문에서는 배치 방식에 대비되는 온라인 모형 적합 방법을 샘플 수가 수천만건에 이르고 다수의 범주형 변수, 그리고 범주의 개수가 유동적인 대규모 데이터에 적용하기 위한 모형 적합 방법과 해싱을 이용한 가변수 코딩(Feature Hashing)에 대해 고려해 보고 각 방법의 특성에 대해서 고찰해 볼 것이다. 또한 가상 데이터와 실제 데이터에에 각 방법론의 적용해 보고 각각의 특성을 실증적으로 살펴볼 것이다. 

## 2. 해싱을 이용한 가변수 코딩(Feature Hashing)

###### 해시?

> - 해시 함수(hash function)는 임의의 길이의 데이터를 고정된 길이의 데이터로 매핑하는 알고리즘.   
> - 암호화에 많이 사용됨

###### 종류

> - 암호화 해시: 복호화가 어려워야 함, (MD5, SHA)
> - 비암호화 해시: 분포적 특성과 해싱 속도가 중요, (murmur, city, spooky)

###### 예시

In [2]:
import mmh3
mmh3.hash128('myCategoryStr')
# producing integer between 0 and 2^128

66459292753059831069357141685600049415L

In [3]:
from spooky import hash128, hash64, hash32
hash32('myCategoryStri')
# producing integer between 0 and 2^32

1472581111L

![Local image](./images/feature_hashing_overview.PNG "Tooltip for local image")

---

> 다수의 변수가 포함되어 있거나 범주형 변수의 범주가 많은 자료를 분석하고자 하는 경우 차원을 축소(Dimensionality reduction)하고 가변수 값의 성김(sparsity)으로 인한 많은 메모리 사용량 문제를 해결하고, 빠르게 데이터 레코드에 해당하는 메모리 상의 feature 벡터의 값에 접근하게 하고, 온라인 학습에서는 매 분석 시점마다 범주형 변수의 범주가 추가될 수도 있는 문제를 해결하기 위해 Hashing Trick(Feature Hashing)을 사용한다.

> 해싱 방식은 크게 암호방식과 비암호방식으로 나뉜다. 암호방식은 해싱 성능 보다는 해시값 복호화가 어려워야 한다는 중요한 특성을 갖도록 고안된 방식이다. 반면 비암호화 방식은 복호과가 가능하느냐는 중요하지 않고 해싱을 수행하는 속도와 해시 결과의 분포적 특성에 중점을 둔 방식이다. 여기서 분포적 특성이란 입력값이 문자열 상으로 유사하더라도 그 결과는 어느 구간에 밀집되지 않고 전체 해시 공간에 고르게 분포하는 것을 말한다.\citep{Ramadhian2013} 속도 측면에서는 대표적인 암호화 해시 방식인 SHA-1이 0.09 bytes/cycle의 해싱 속도를 갖는 반면 비암호화 해시의 하나인 MurmurHash의 경우 1 byte/cycle의 속도로서 거의 10배 빠르다. 

> 물론 Feature Hashing을 사용하는 것이 장점만이 있는 것은 아니다. 대표적인 문제로 해시 충돌을 들 수 있다. 분포도가 양호한 해시 방식을 사용하는 것이 가장 우선 이겠으나 해시 충돌은 발생할 수 있는 문제이고, 이를 줄이기 위해 해시 함수에 입력될 문자열을 모든 변수의 범주를 통틀어 가능한 유일 하도록 매핑하는 작업을 하거나 피쳐 벡터(feature vector)의 크기를 고려하여 해시 테이블 크기를 조정 하는 것도 한 방법이다.

> 구체적으로 해싱을 가변수(dummy variable) 설정에 어떻게 사용되는지 살펴보자. 샘플 수가 $N$인 가변수 생성 전 데이터에서 범주형인 $j$번째 변수 $x_j$에 $C_1 , C_2 , \cdots , C_{K_j}$의 $K_j$개 범주가 있다고 해 보자. 일반적인 가변수 코딩에서는 $K_j$개의 범주에 대해 $K_j - 1$개의 가변수를 설정하는데 그럴 경우 각 가변수 벡터는 $K_j - 2$개의 0과 1개의 1로 구성된다. 그런데 범주의 수 $K_j$가 커지고 샘플 수가 많아지면 가변수 매트릭스의 성김(sparsity)이 심해지고 정보량에 비해 메모리 사용량과 처리 부하가 커지게 된다. 또한 스트리밍으로 유입되는 새로운 데이터에서 $x_j$에 기존에 없던 범주($C_{K_j + 1}$)가 등장할 경우 가변수 설정과 적합을 다시 진행해야 한다. 
 반면 Feature Hashing을 적용할 경우를 살펴보자. $x_j$의 $k(0 \leq k \geq K_j)$번째 범주 $C_k$를 문자열 그대로 혹은 전체 범주에 대한 유일성을 보장하기 위해 $x_j C_k$를 문자열 형태로 해시 함수의 입력으로 넣는다. 예를들어 64비트 해시 함수를 사용할 경우 1과 $2^{64}$ 사이의 숫자($h_jk$)를 출력 값으로 얻게 된다. 이 숫자는 모수 벡터의 인덱스를 의미하기 때문에 $i$번재 샘플의 $j$번째 변수 $x_j$는 0과 1로 이루어진 벡터가 아니라 단순히 숫자 $h_ijk$로 표현된다. 만약 변수 변환 이후 대략적인 전체 모수 벡터의 길이가 $2^64$보다 월등히 작을 경우 적당한 숫자 $L$로 $h_ijk$를 모듈러 연산하여 모수 벡터의 길이를 $L$로 제한할 수 있다. 즉 $L$이 충분히 클 경우 대략 $L$개까지 범주 수가 늘어나더라도 가변수를 다시 설정하지 않아도 된다.

## 3. 온라인 최적화 알고리즘

### 3.1. 확률적 경사 하강법(Stochastic Gradient Descent)

> 최적화(optimization) 알고리즘의 하나인 경사하강법(Gradient descent)에서는 전체 샘플 데이터를 스캔 할 때마다 회귀 계수 추정치를 갱신해 나간다. 비용함수(cost function)를 $J(w) = \frac{1}{2} \sum_{i}(target^{(i)} - output^{(i)})^{2}$ 라 할때 길이가 j인 계수 벡터(weight vector) $w_i$를 $w_{i+1}=w_{i}+\Delta w$로 갱신하는 매 반복에서 $j$개의 모수 추정치($w$)를 얼마만큼 줄일지 혹은 늘릴지 $\Delta w$ 값을 결정해야 한다. 여기에서 $\Delta w$는 아래와 같다.
\begin{eqnarray}
\Delta w_{j} 	&=& -\alpha \frac{\delta J}{\delta w_{j}} \\
							&=& -\alpha \sum_{i}(target^{(i)} - output^{(i)})(-x^{(i)_{j}}) \\
							&=& \alpha \sum_{i}(target^{(i)} - output^{(i)})(x^{(i)_{j}})
\end{eqnarray}
즉 비용함수(cost function)의 경사도가 크면 그만큼 많이 $w$값을 수정하게 되는데, 학습 계수(learning rate, step size) $\alpha$에 비례하여 수정치가 결정된다.

> 경사하강법에서는 한번 $w$값을 수정하기 위하여 전체 샘플 데이터를 스캔하게 된다. 그런데 이런 방식은 샘플의 수가 많은 경우 
비용함수 $J(w)$를 최소로 하는 $w$값을 찾기 까지 그 처리 시간이 길어 지게 된다. 모수에 대한 학습(learning)이나 추론(inference)를 진행할 때 데이터의 크기가 작거나 데이터의 수집으로부터 예측까지 시간적 여유가 있을 경우 데이터 전체를 한꺼번에 활용하는 일괄 처리(batch processing) 방식을 사용한다. 반면 데이터를 한꺼번에 처리하기에 그 크기가 지나치게 크거나 스트리밍(streaming)으로 유입되는 데이터에 대해서 실시간으로 예측을 처리해야 하는 경우 온라인 학습(online learning)을 사용해야 할 필요성이 있다.

> 이처럼 매 갱신에서 전체 샘플 데이터를 사용하는 대신 샘플 하나 혹은 일부분만을 사용하여 $w$ 값을 갱신해 가는 경사하강법을 확률적 경사 하강법(Stochastic Gradient Descent, Online Gradient Descent)이라 하며, 확률적 이라는 단어에서 알 수 있듯이 $J(w)$를 최소로 하는 $w$값을 확률적 근사방법으로 찾아가는 것이다. 하나의 샘플만을 이용해 $w$의 갱신 방향과 크기를 결정하기 때문에 경사하강법과는 달리 $J(w)$값이 커지는 경우도 있으나 샘플 수가 충분할 경우 $J(w)$의 전역 최소값으로 수렴하는 $w$를 찾을 수 있는 것으로 알려져 있다.

> - 본 연구에서는 전체 파라미터 벡터에 동일한 step size 초기값을 설정하고, iteration이 증가하면서 각 hashed-feature가 등장하는 횟수로 step size를 나눈다.  
> - 즉 특정 hashed-feature가 많이 등장할 수록 해당 회귀 모수에 대한 step-size가 감소하게 되고, 자주 등장하지 않는 hashed-feature의 경우 비교적 큰 step-size를 갖게 된다.

###### [알고리즘]

In [44]:
# simplified version of SGD
c_fi = fi_titanic

D = 2 ** 20
alpha = 0.92000539999999997 # initial value for setp size alpha
w = [0.] * D  # parameter vector
n = np.array([0.] * (D))

f = open(c_fi.file_path)
for t, row in enumerate(DictReader(f, fieldnames=c_fi.l_header_names, delimiter=c_fi.seperator)):  # for titanic(comma seperated)
    if t == 0:
        continue
    del row['PassengerId']
    
    y = 1. if row[c_fi.ylab] == '1' else 0.
    del row[c_fi.ylab]
    
    x = get_x_mmh3(row, D)
    p = get_p(x, w)
    w, n = update_w_withn(w, n, x, p, y, alpha)
f.close()

In [ ]:
def update_w_withn(w, n, x, p, y, alpha):
    for i in x:
        w[i] -= (p - y) * alpha / (sqrt(n[i]) + 1.)
        n[i] += 1.
    return w, n

### 3.2. 추정된 밀도 필터링(Assumed Density Filtering)

###### 추정된 밀도 필터링(Assumed-density filtering, ADF) 이란?

> 추정된 밀도 필터링(Assumed-density filtering, ADF)는 베이지안 네트워크 혹은 여타의 통계 모형에서 사후분포를 근사적으로 계산하는 방법으로서 통계학에는 Lauritzen, 1992이 제안. 분야에 따라 달리 불림
 
 > - 추정된 밀도 필터링(Assumed-density filtering)  
 > - 온라인 베이지안 학습(On-line Bayesian learning)  
 > - 적률 대응(Moment matching)  
 > - 약한 주변화(Weak marginalization)  

>  ADF에서는 사후분포를 가우시안과 같은 특정 분포로 근사하는 방법으로서 예측-갱신-투영(predict-update-project)과정을 반복한다. 
 
> - 예측(predict): 모수 $\theta$에 대한 $t-1$ 시점의 사전분포, $q_{t-1}(\theta_{t-1})$와 $t$시점의 관측치를 이용하여 이후 시점 $t$에서의 $\theta$에 대한 사후예측분포, $q_{t|t-1}(\theta_{t})$를 구함
> - 갱신(update): 앞서 구한 사전분포와 사후예측분포를 이용하여 $\theta$에 대한 사후분포, $\hat{p}(\theta_t)$를 구함 
> - 투영(project): 마지막으로 이 사후 분포가 다루기 쉬운 형태가 아닌 경우가 빈번하기 때문에 다루기 쉬운 분포로 투영(project)

> - 근사 사전분포: 
$$q_{t-1}(\theta_{t-1}) \approx p(\theta_{t-1}|y_{1:t-1})$$
<br>
- 1단계 사후예측분포: 
$$q_{t|t-1}(\theta_t) = \int p(\theta_t | \theta_{t-1}) q_{t-1}(\theta_{t-1}) d\theta_{t-1}$$
<br>
- 사후분포:
$$\hat{p}(\theta_t) = \frac{1}{Z_t}p(y_t | \theta_t)q_{t|t-1}(\theta_t)$$
<br>
- 정규화 상수(normalizing constant):
$$Z_t = \int p(y_t | \theta_{t-1})q_{t|t-1}(\theta_{t})d\theta_{t}$$
<br>
- 근사 사후분포:
$$q(\theta_t) = \arg\min_{q \in Q} \mathrm{KL}(\hat{p}(\theta_t || q(\theta_t)) $$

###### 지수족에서의 추정된 밀도 필터링(Assumed-density filtering, ADF) 

> 투영하려는 분포 $q$가 지수족에 속할 경우 단순히 적률 대응(moment matching)만으로 $q(\theta_t)$를 구할수 있다. 따라서 일반화 선형모형에 적률 대응을 적용.

> 편의를 위해 설명변수와 회귀계수의 선형식(systematic component)을 $s_t=\theta_t^T x_t$라 하고, 만약 $\theta_t$에 대한 1단계 사후예측분포, $q_{t|t-1}(\theta_t)$가$\prod_i N(\theta_{t,i};\mu_{t|t-1,i},\sigma^2_{t|t-1,i})$라면 $s_t$의 사후 예측분포, $q_{t|t-1}(s_t)$ 는 아래와 같다.

> \begin{eqnarray}
   q_{t|t-1}(s_{t}) &\equiv& N(s_t;m_{t|t-1}, {v}_{t|t-1})
\\ m_{t|t-1} &=& \sum^N_{i=1}x_{t,i}\mu_{t|t-1,i}
\\ {v}_{t|t-1} &=& \sum^N_{i=1}x^2_{t,i}{\sigma}^2_{t|t-1,i}
\end{eqnarray}

> 이때 $s_t$의 사후분포, $q_t(s_t)$는 아래와 같다.

> \begin{eqnarray}
 q_t(s_t) &\equiv& N(s_t; m_t, v_t)
 \\ m_t &=& \int s_t \frac{1}{z_t} f(y_t|s_t) q_{t|t-1}(s_t)ds_t
 \\ v_t &=& \int s^2_t \frac{1}{z_t} f(y_t|s_t) q_{t|t-1}(s_t) ds_t - m_t^2  
\\ z_t &=& \int f(y_t|s_t) q_{t|t-1}(s_t)ds_t
\\ f(y_t|s_t) &\equiv& Ber(y_t;\pi = sigmoid(s_t))
\\ & =& \pi^{y_t} (1-\pi)^{(1-y_t)}, \quad y_t \in \{0,1\}
\end{eqnarray}

> 가우시안 구적법(Gaussian quadrature)을 이용해 위 적분식 근사
$$\int^b_a W(x)f(x)dx \approx \sum^N_{j=1}w_j f(x_j)$$
$$\chi=\chi'\sqrt{2}\sigma_{s_t}+\mu_{s_t}$$ 
$$\omega_i = \frac{\omega'}{\sqrt{\pi}}$$

> \begin{eqnarray}
q_t(s_t) &=& N(s_t; \tilde{m}_t, \tilde{v}_t)
\\ \tilde{m}_t &=& \frac{1}{\tilde{z}_t} \sum_i \chi_i f(y_t; \chi_i ) \omega_i
\\ \tilde{v}_t &=& \frac{1}{\tilde{z}_t} \sum_i \chi^2_i f(y_t; \chi_i ) \omega_i - \tilde{m}^2_t
\\ \tilde{z}_t &=& \sum_i f(y_t; \chi_i ) \omega_i
\end{eqnarray}

###### 알고리즘

In [ ]:
init_v = 0.52007399999999993 # best for titanic

theta_t_m = np.array([0.] * (D)) # mean of thetas at t
theta_t_v = np.array([init_v] * (D)) # variance of thetas at t
n = np.array([0.] * D)

f = open(c_fi.file_path)
for t, row in enumerate(DictReader(f, fieldnames=c_fi.l_header_names, delimiter=c_fi.seperator)):

    if t == 0:
        continue
    del row['PassengerId']
    
    y = 1. if row[c_fi.ylab] == '1' else 0.
    del row[c_fi.ylab]

    x = get_x_mmh3(row, D)

    # Predictive distribution for s_t ~ N(s_t_m_old, s_t_v_old)
    s_t_m_old = sum(theta_t_m[x])
    s_t_v_old = sum(theta_t_v[x])

    # Posterior distribution for s_t
    s_t_m, s_t_v = get_s_t_new(y, s_t_m_old, s_t_v_old)

    # Changes in s_t
    delta_m = s_t_m - s_t_m_old
    delta_v = s_t_v - s_t_v_old

    # Updating theta
    update_theta_cat(x, theta_t_m, theta_t_v, delta_m, delta_v, t, n)

    #p = get_p_cat(x, theta_t_m)

f.close()

In [ ]:
# s_t_m_old and s_t_v_old must be numpy ndarray
def get_s_t_new(y, s_t_m_old, s_t_v_old):

    wi = wwi / np.sqrt(np.pi)
    xi = xxi * np.sqrt(2) * np.sqrt(s_t_v_old) + s_t_m_old
    
    fw = 0.
    if(y==1):
        fw = (1. / (1. + np.exp(-xi))) * wi
    else:
        fw = ((np.exp(-xi)) / (1. + np.exp(-xi))) * wi

    z_t = sum(fw)
    s_t_m_new = 1. / z_t * sum(xi * fw)
    s_t_v_new = 1. / z_t * sum((xi**2) * fw) - s_t_m_new**2
        
    return (s_t_m_new, s_t_v_new)

In [ ]:
def update_theta_cat(x, theta_t_m, theta_t_v, delta_m, delta_v, n_iter, n):
    a_i = get_a_i_cat(x, theta_t_v)
    theta_t_m[x] += (a_i * delta_m)
    theta_t_v[x] += ((a_i**2) * delta_v)
    n[x] += 1.

In [ ]:
# theta_t_v must be numpy ndarray
def get_a_i_cat(x, theta_t_v):
    return theta_t_v[x] / sum(theta_t_v[x])

## 4. 실험

### 4.1. 실험(타이타닉 데이터)

###### 개요

> - 소규모 데이터를 이용한 실험을 위해 891건의 타이타닉 생존자 데이터를 이용  
> - 데이터는 생존여부(0, 1)를 나타내는 'Survived'와 탑승자의 특성을 나타내는 8가지 변수로 구성됨.
> - 초기 800건을 훈련 데이터, 나머지 91건을 테스트 데이터로 사용

###### 최적 초기값

> SGD는 step size의 초기값($\alpha_{init}$)에, ADF는 회귀 계수 분포의 초기 분산값($V_{init}$)에 큰 영향을 받음.  
> 주어진 데이터에 최적인 $\alpha_{init}$과 $V_{init}$를 찾기 위해 각 값을 변화시키면서, 훈련 데이터와 테스트 데이터에 대한 log-loss를 관찰함

> $logloss = -\frac{1}{N}\sum_{i=1}^N {(y_i\log(p_i) + (1 - y_i)\log(1 - p_i))}$

<center>**Stochastic Gadient Descent**</center>

 ![Local image](./images/best_param_sgd_T.png "Tooltip for local image")

|          | Index     | $\alpha$          |  log-loss |
|----------|:-------:|:--------------:|
| log-loss      | 11 | 0.11000779999999999 | 0.46879230769345764

>  테스트 데이터에 최소 log-loss를 갖는 $\alpha$값은 0.11000779999999999

<center>**Assumed Density Filtering**</center>

 ![Local image](./images/best_param_adf_T.png "Tooltip for local image")

|          | Index     | $\alpha$          |  log-loss |
|----------|:-------:|:--------------:|
| log-loss      | 13 | 0.52007399999999993 | 0.39539652671035114

> 테스트 데이터에 최소 log-loss를 갖는 분산값은 0.52007399999999993

###### 훈련 데이터 크기에 따른 log-loss 비교

> 16건의 훈련 데이터를 더 사용할 때마다 그때의 회귀 계수 벡터를 이용하여 91건의 테스트 데이터에 대한 log-loss 추이를 두 방법론(SGD, ADF)에 대해 비교한다.

 ![Local image](./images/step_vali_comparison_T.png "Tooltip for local image")

> 전반적으로 ADF에 비해 SGD의 log-loss가 낮음.

 ![Local image](./images/step_vali_roc_T.png "Tooltip for local image")

|          | SGD     | ADF          |
|----------|:-------:|:--------------:|
| AUC      |0.90041279669762642 | 0.88802889576883381  |

### 4.2. 실험(온라인 광고 데이터)

###### 개요

> Criteo에서 'Kaggle 대회'를 위해 공개한 4천 5백만건 상당의 온라인 광고 데이터를 사용하였다. 데이터는 웹사이트 방문자가 해당 광고를 클릭 했으냐 혹은 하지 않았느냐를 나타내는 이항 반응변수 Label과 39개의 설명변수로 구성되어 있다. 그리고 각 설명변수는 범주형으로서 범주는 500개 이상으로 실제 데이터의 경우 훈련 데이터에 없던 범주가 새롭게 등장 할 수도 있는 특성을 갖는다. 이러한 데이터를 일괄처리(batch) 방식으로 처리한다면 대략 $19,500 \times 45,000,000$ 크기의 매트릭스 연산을 수행해야 한다.

###### 최적 초기값

<center>**Stochastic Gadient Descent**</center>

 ![Local image](./images/best_param_sgd_C.png "Tooltip for local image")

<center>**Assumed Density Filtering**</center>

 ![Local image](./images/best_param_adf_C.png "Tooltip for local image")

###### 훈련 데이터 크기에 따른 log-loss 비교( 훈련 데이터: 1만건, 테스트 데이터 1천건)

 ![Local image](./images/step_vali_comparison_C.png "Tooltip for local image")

 ![Local image](./images/step_vali_roc_C.png "Tooltip for local image")

|          | SGD     | ADF          |
|----------|:-------:|:--------------:|
| AUC      | 0.75874125874125875 | 0.75699300699300698 |

###### 훈련 데이터 크기에 따른 log-loss 비교 ( 훈련 데이터: 100만건, 테스트 데이터 10만건)

 ![Local image](./images/step_vali_comparison_CC.png "Tooltip for local image")

 ![Local image](./images/step_vali_roc_CC.png "Tooltip for local image")

|          | SGD     | ADF          |
|----------|:-------:|:--------------:|
| AUC      | 0.78110714529429526 | 0.78198942683050354|



## 5. 맺음말

> 본 논문에서는 다변수 대용량 데이터에 대한 실시간 모형 적합 방법과 구현시 고려 사항에 대해 고찰해 보았다. 우선 데이터의 규모가 커질 경우 배치 방식의 한계점으로 인하여 온라인 방식의 모형 적합 방법을 적용해야 했고, 대표적인 온라인 모형 적합 방법인 확률적 경사 하강법(Stochastic Gradient Descent)과 이 방법의 베이지안 접근이라 할 수 있는 추정된 밀도 필터링(Assumed-Density Filtering) 방법을 사용하여 예측 결과의 추이를 살펴보았다. 

> 일반적으로 베이지안 접근 방법은 대규모 데이터 처리에 적합하지 않다는 주장이 많다. 하지만 추정된 밀도 필터링과 같은 근사 분포를 사용하는 방법론을 사용할 경우 실시간 처리가 필수적인 온라인 러닝에 있어서도 충분히 만족스러운 속도로 분석을 진행 할 수 있음을 확인 할 수 있었다. 또한 컴퓨팅 속도를 향상 시킬 수 있는 프로그래밍 언어, GPU 컴퓨팅, 분산 처리 기법등을 적용할 경우 방법론 선택에 대한 제약은 크지 않다고 할 수 있다.

> 실제로 동일한 알고리즘을 R과 Pthon을 통해 모두 구현해 보았는데 절대적으로 Python을 이용한 구현이 실행속도 면에서 압도적이었다. 4500만건 트레이닝 상황에서 R을 이용한 구현에서는 대략 2주정도 실행 시간이 예상되었으나 Python을 이용한 구현에서는 불과 2시간만에 결과를 얻을 수 있었다. 물론 구현 방식과 사용하는 패키지에 따라서 차이가 있겠으나 빠른 실행 시간이 필요한 상황에서는 어떤 프로그래밍 언어를 선택하느냐가 중요한 요소일 수 있다.

> 모수 벡터 설정 및 변수 코딩과 관련하여 다범주 다변수 데이터를 실시간 분석해야할 경우 해싱을 이용한 접근 방법을 사용하는 것이 효과적임을 확인할 수 있었다. 우선 단순히 차원 축소 뿐만 아니라 각 셈플 벡터의 크기가 줄어 메모리 사용량이 줄어들고 모수 벡터에 대한 빠른 접근이 가능하여 실행 성능 향상 효과를 얻을 수 있었고 대략적인 변수 규모만 지정하면 범주가 추가될 경우에도 다시 가변수 코딩이나 적합을 수행할 필요가 없는 장점을 확인 할 수 있었다.

>  확률적 경사 하강법의 경우 step size $\alpha$를 어떻게 조절 하느냐가 예측 결과에 아주 큰 영향을 주었다. $\alpha$를 반복횟수에 반비례하여 조절할 경우 우수한 예측 결과를 얻을 수 있었으나 $\alpha$의 초기값에 따라 모형의 성능이 크게 좌우 되었다. 반면 추정된 밀도 필터링의 경우 각 모수 분포의 분산($\sigma$) 초기값에 따라 모형의 성능이 변할 수 있음을 확인 할 수 있었다. 또한 $\sigma$값에 따라 확률적 경사하강법에 비해 초기에 낮은 log-loss값을 보여주기도 했으나, 적합 반복이 증가할 경우 두 모형 사이에 절대적인 우위를 확인할 수는 없았다. 결국 확률적 경사 하강법에서는 step size $\alpha$, 추정된 밀도 필터링에서는 각 모수 분포의 분산($\sigma$) 초기값을 적절히 설정하느냐가 모형의 성능을 결정하는 중요한 요소임을 알 수 있었다.


---